## Requirements

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- 30% of the company have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not to far.
- Account managers need to travel a lot
- All people in the company have between 25 and 40 years, give them some place to go to party.
- Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.
- The CEO is Vegan

In [1]:
from pymongo import MongoClient
import folium
import pandas as pd

Function to conect to the database and collection of MongoDB in localhost:

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

Function to get the latitude and longitude in geoJSON format:

In [3]:
db, coll = connectCollection('companies','companies')

In [4]:
def getLocation(comp,i):
    latitude = comp['offices'][i]['latitude']
    longitude = comp['offices'][i]['longitude']
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

In [15]:
def getName(comp,i):
    name = comp['name']
    return name

Let's find all the companies that have less than 10 years since their foundation:

In [24]:
company = coll.find({"$and": [{"total_money_raised": {"$gt": "$1M"}},{"founded_year": {"$gt":2009}}]})

Now I set a new field in the documents, call "location", which will contain the info provided by getLocation function:

In [26]:
for c in company:
    for i in range(len(c['offices'])):
        value = {"$set": {'offices.'+str(i)+'.location':getLocation(c,i)}}
        coll.update_one(c,value)



In [25]:
for c in company:
    for i in range(len(c['offices'])):
        value = {"$set": {'location.'+str(i):getLocation(c,i)}}
        coll.update_one(c,value)



In [17]:
for c in company:
    for i in range(len(c['offices'])):
        value = {"$unset": {'location'+str(i):''}}
        coll.update_one(c,value)

In [10]:
company = coll.find({"$and": [{"total_money_raised": {"$gt": "$1M"}},{"founded_year": {"$gt":2009}}]})

In [6]:
df = pd.DataFrame(list(company))

In [8]:
new_df = df[['name','founded_year','description','offices']]

In [9]:
new_df

,name,founded_year,description,offices
0,AdventNet,1996,Server Management Software,"[{'description': 'Headquarters', 'address1': '..."
1,Wetpaint,2005,Technology Platform Company,"[{'description': '', 'address1': '710 - 2nd Av..."
2,Zoho,2005,Online Business Apps Suite,"[{'description': 'Headquarters', 'address1': '..."
3,Omnidrive,2005,None,"[{'description': '', 'address1': 'Suite 200', ..."
4,Postini,1999,None,"[{'description': None, 'address1': '959 Skyway..."
...,...,...,...,...
12298,Contemporary Computer Services,1974,,"[{'description': '', 'address1': '200 Knickerb..."
12299,Eazeeloans,2006,Loan advisory portal,"[{'description': 'Apex Finance & Marketing', '..."
12300,AfterLogic,2002,.NET Email Components and Webmail,"[{'description': 'Livingston', 'address1': 'P...."
12301,EnteGreat,2000,,"[{'description': '', 'address1': '1900 Interna..."


In [18]:
company = coll.find({"founded_year": {"$lt":2009}})

In [ ]:
df2 = pd.DataFrame(columns=['name'])

In [19]:
for c in company:
        for i in range(len(c['offices'])):
            value = {"$set": {'offices.'+str(i)+'.name':getName(c,i)}}
            coll.update_one(c,value)
    

## APIs

In [26]:
import os
from dotenv import load_dotenv
import requests
load_dotenv()


True

In [30]:
def googleRequestAuthorized():

    # Function
    authToken = os.getenv("GOOGLE_API_TOKEN")
    if not authToken:
        raise ValueError("NECESITAS UN TOKEN")
    url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Vegan%20Restaurant%20Madrid%20Spain&inputtype=textquery&fields=formatted_address,name,geometry&key={}".format(authToken)

    res = requests.get(url)
    data = res.json()
    return data

In [31]:
example = googleRequestAuthorized()

In [32]:
print(example)

{'candidates': [{'formatted_address': 'Calle de la Luna, 9, 28004 Madrid, España', 'geometry': {'location': {'lat': 40.422335, 'lng': -3.706148}, 'viewport': {'northeast': {'lat': 40.42368492989272, 'lng': -3.704753870107278}, 'southwest': {'lat': 40.42098527010728, 'lng': -3.707453529892722}}}, 'name': 'VEGA', 'opening_hours': {'open_now': False}, 'rating': 4.5}], 'status': 'OK'}
